In [1]:
# Import packages
import pandas as pd
import numpy as np
import pyodbc
import warnings
from datetime import datetime
from datetime import timedelta

warnings.filterwarnings("ignore")

In [2]:
# Query CNX hourly average load and corresponding weather

query = f'''
        select *

	from

		((select 
			c.intervalStart, 
			avg(c.Connexus_kWh) Connexus_kWh
		
			from (select 
					dateadd(hour, -1, dateTime) intervalStart, 
					kwh Connexus_kWh
			
				from [GRE_EDW].[cnx_custom].[GREMVWebData]

				union all

				select 
					intervalStart, 
					averageValue * 1000 Connexus_kWh

				from SCADA_EDWSTG.cnx_custom.scadaChronusTimeseriesAggregate 

				where 
					displayKey = '04150001' and 
					intervalStart >= (select max(datetime) from [GRE_EDW].[cnx_custom].[GREMVWebData])) c

		
		group by c.intervalStart) t1

		inner join 
		
		(select *
		from WEATHER_EDW.dbo.vw_weatherHistory
		where station = 'STF ') t2																																													
		
		on t1.intervalStart = t2.dateTime)

	order by intervalStart asc'''

with pyodbc.connect('dsn=WEATHER_EDW') as conn:
            
            load_weather_df = pd.read_sql(query, conn)
            
# Query GRE load management program history

query = f'''
        SELECT
	convert(date, eventStartTime) date
	,[eventStartTime] ProgramStart
	,[eventEndTime] ProgramEnd
	,[DRProgramDescription] Program
FROM [PTR_EDW].[cnx_custom].[demandResponseEventGRE]'''

with pyodbc.connect('dsn=WEATHER_EDW') as conn:
            
            GRE_programs_df = pd.read_sql(query, conn)
            
# Query CNX load management program history

query = f'''
        SELECT
	convert(date, ProgramStart) date
	,[PSOScheduleID]
	,[Program]
	,[ProgramStart]
	,[ProgramEnd]
	,[Notes]
	,[createDate]
	,[updateDate]
	FROM [EDW].[PSO].[tPSOProgramSchedule]'''

with pyodbc.connect('dsn=WEATHER_EDW') as conn:
            
            CNX_programs_df = pd.read_sql(query, conn)

#### Data Initial Observations - Load+Weather

In [3]:
load_weather_df.head()

,intervalStart,Connexus_kWh,unixTime,dateTime,latitude,longitude,station,apparentTemperature,cloudCover,dewPoint,...,precipType,pressure,snowAccumulation,snowIntensity,temperature,uvIndex,visibility,windBearing,windGust,windSpeed
0,2017-01-01 01:00:00,194634.4219,1483254000,2017-01-01 01:00:00,45.395556,-93.386667,STF,18.93,0.39,19.23,...,None,29.79,NaN,NaN,25.48,0.0,10.0,224.0,10.51,5.48
1,2017-01-01 02:00:00,185003.5234,1483257600,2017-01-01 02:00:00,45.395556,-93.386667,STF,20.16,0.00,18.98,...,None,29.81,NaN,NaN,25.55,0.0,10.0,231.0,8.88,4.40
2,2017-01-01 03:00:00,179523.5938,1483261200,2017-01-01 03:00:00,45.395556,-93.386667,STF,20.40,0.04,18.58,...,None,29.83,NaN,NaN,24.89,0.0,10.0,235.0,6.34,3.59
3,2017-01-01 04:00:00,178223.6797,1483264800,2017-01-01 04:00:00,45.395556,-93.386667,STF,24.46,0.20,18.62,...,None,29.85,NaN,NaN,24.46,0.0,10.0,245.0,5.83,2.88
4,2017-01-01 05:00:00,180018.2031,1483268400,2017-01-01 05:00:00,45.395556,-93.386667,STF,23.61,0.11,18.69,...,None,29.88,NaN,NaN,23.61,0.0,10.0,238.0,3.13,1.66


In [4]:
print(load_weather_df.shape)
print(load_weather_df.drop_duplicates().shape)

(58576, 25)
(58576, 25)


There is no duplicates in load_weather dataset

In [5]:
# datetime type features
load_weather_df[['intervalStart', 'dateTime']].describe()

,intervalStart,dateTime
count,58576,58576
unique,58570,58570
top,2021-11-07 01:00:00,2021-11-07 01:00:00
freq,2,2
first,2017-01-01 01:00:00,2017-01-01 01:00:00
last,2023-09-25 12:00:00,2023-09-25 12:00:00


In [6]:
# categorical type features
load_weather_df.describe(include=object)

,station,icon,precipType
count,58576,58572,16095
unique,1,30,3
top,STF,clear-night,rain
freq,58576,9371,10704


In [7]:
# numeric type features
load_weather_df.describe()

,Connexus_kWh,unixTime,latitude,longitude,apparentTemperature,cloudCover,dewPoint,humidity,precipAccumulation,precipIntensity,precipProbability,pressure,snowAccumulation,snowIntensity,temperature,uvIndex,visibility,windBearing,windGust,windSpeed
count,58576.000000,5.857600e+04,5.857600e+04,5.857600e+04,58576.000000,58562.000000,58576.000000,58576.000000,18067.000000,58563.000000,58563.000000,58375.000000,13598.000000,13593.000000,58576.000000,58568.000000,58567.000000,58317.000000,58553.000000,58576.000000
mean,235894.490105,1.589030e+09,4.539556e+01,-9.338667e+01,42.194704,0.468365,35.946073,0.722284,0.012206,0.002846,0.065281,29.997937,0.009361,0.001055,45.436708,1.010330,9.356314,195.884562,12.496694,6.865357
std,61076.475321,6.131410e+07,2.685874e-11,6.251408e-11,27.802182,0.404621,21.696214,0.176015,0.053890,0.021105,0.208396,0.243699,0.063410,0.008575,23.926283,1.853823,1.724108,102.731832,7.862491,4.385376
min,130035.377000,1.483254e+09,4.539556e+01,-9.338667e+01,-50.390000,0.000000,-70.960000,0.000000,0.000000,0.000000,0.000000,28.970000,0.000000,0.000000,-31.660000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,196783.495561,1.535979e+09,4.539556e+01,-9.338667e+01,21.577500,0.040000,21.210000,0.610000,0.000000,0.000000,0.000000,29.840000,0.000000,0.000000,28.190000,0.000000,9.940000,119.000000,6.570000,3.780000
50%,225613.376394,1.588711e+09,4.539556e+01,-9.338667e+01,43.925000,0.400000,35.870000,0.760000,0.000000,0.000000,0.000000,29.980000,0.000000,0.000000,46.050000,0.000000,10.000000,194.000000,11.160000,6.110000
75%,259491.117175,1.641543e+09,4.539556e+01,-9.338667e+01,65.680000,0.930000,54.900000,0.860000,0.000000,0.000100,0.010000,30.140000,0.000000,0.000000,65.530000,1.000000,10.000000,291.000000,17.110000,9.320000
max,542499.336667,1.695661e+09,4.539556e+01,-9.338667e+01,112.150000,1.000000,79.130000,1.000000,1.570000,1.340000,1.000000,30.930000,1.820000,0.330000,98.380000,10.000000,10.000000,360.000000,49.630000,29.620000


In [8]:
# check null values
for column in load_weather_df.columns:
    print(column, load_weather_df[column].isnull().sum())

intervalStart 0
Connexus_kWh 0
unixTime 0
dateTime 0
latitude 0
longitude 0
station 0
apparentTemperature 0
cloudCover 14
dewPoint 0
humidity 0
icon 4
precipAccumulation 40509
precipIntensity 13
precipProbability 13
precipType 42481
pressure 201
snowAccumulation 44978
snowIntensity 44983
temperature 0
uvIndex 8
visibility 9
windBearing 259
windGust 23
windSpeed 0


Null value will be replaced with model learnable value after aggregating data.

#### Data Initial Observations - GRE program history

In [9]:
GRE_programs_df.head()

,date,ProgramStart,ProgramEnd,Program
0,2018-03-07,2018-03-07 17:00:00,2018-03-07 21:00:00,Interruptible Water Heating
1,2018-03-06,2018-03-06 17:00:00,2018-03-06 21:00:00,Interruptible Water Heating
2,2018-02-01,2018-02-01 16:30:00,2018-02-01 21:30:00,Interruptible Water Heating
3,2018-01-15,2018-01-15 16:00:00,2018-01-15 21:30:00,Interruptible Water Heating
4,2018-01-05,2018-01-05 16:00:00,2018-01-05 21:30:00,Interruptible Water Heating


In [10]:
# check duplicates
print(GRE_programs_df.shape)
print(GRE_programs_df.drop_duplicates().shape)

(1403, 4)
(446, 4)


There are many duplicates. We need to leave the one unique.

In [11]:
# drop duplicates
GRE_programs_df = GRE_programs_df.drop_duplicates()
print(GRE_programs_df.shape)

(446, 4)


In [12]:
# check null values
for column in GRE_programs_df.columns:
    print(column, GRE_programs_df[column].isnull().sum())

date 0
ProgramStart 0
ProgramEnd 0
Program 0


In [13]:
GRE_programs_df.describe()

,date,ProgramStart,ProgramEnd,Program
count,446,446,446,446
unique,220,378,370,13
top,2021-02-08,2021-02-08 06:30:00,2018-03-07 19:00:00,Interruptible Water Heating
freq,6,4,6,210
first,NaN,2018-01-03 16:00:00,2018-01-03 21:00:00,NaN
last,NaN,2023-09-05 14:00:00,2023-09-05 19:00:00,NaN


The earlist date of the column 'ProgramStart' is 2018-01.03, so we might have to cut the time-series data before the date.

#### Data Initial Observations - CNX program history

In [14]:
CNX_programs_df.head()

,date,PSOScheduleID,Program,ProgramStart,ProgramEnd,Notes,createDate,updateDate
0,2023-01-06,1,DVR,2023-01-06 16:45:00,2023-01-06 20:10:00,None,2023-07-27 16:11:27,2023-07-27 16:11:27
1,2023-01-06,2,CampusGen,2023-01-06 16:55:00,2023-01-06 20:00:00,None,2023-07-27 16:11:27,2023-07-27 16:11:27
2,2023-01-29,3,DVR,2023-01-29 16:45:00,2023-01-29 20:10:00,None,2023-07-27 16:11:27,2023-07-27 16:11:27
3,2023-01-29,4,CampusGen,2023-01-29 16:55:00,2023-01-29 20:00:00,None,2023-07-27 16:11:27,2023-07-27 16:11:27
4,2023-01-30,5,DVR,2023-01-30 05:45:00,2023-01-30 09:10:00,Morning,2023-07-27 16:11:27,2023-07-27 16:11:27


In [15]:
print(CNX_programs_df.shape)
print(CNX_programs_df.drop_duplicates().shape)

(119, 8)
(119, 8)


No duplicates

In [16]:
# check null values
for column in CNX_programs_df.columns:
    print(column, CNX_programs_df[column].isnull().sum())

date 0
PSOScheduleID 0
Program 0
ProgramStart 0
ProgramEnd 0
Notes 112
createDate 0
updateDate 0


In [17]:
CNX_programs_df[['ProgramStart', 'ProgramEnd']].describe()

,ProgramStart,ProgramEnd
count,119,119
unique,72,75
top,2023-08-24 15:00:00,2023-06-22 18:00:00
freq,4,4
first,2023-01-06 16:45:00,2023-01-06 20:00:00
last,2023-08-24 15:00:00,2023-08-24 19:00:00


In [18]:
CNX_programs_df.describe(include=object)

,date,Program,Notes,createDate,updateDate
count,119,119,7,119,119
unique,42,5,3,9,9
top,2023-07-26,DVR,Morning,2023-07-27 16:11:27,2023-07-27 16:11:27
freq,5,45,4,92,92


In [19]:
CNX_programs_df.describe()

,PSOScheduleID
count,119.000000
mean,60.000000
std,34.496377
min,1.000000
25%,30.500000
50%,60.000000
75%,89.500000
max,119.000000


#### Data pre-processing: aggregate datasets

In [20]:
# transform daily program history data to hourly data
def expand_program_df(sample):


    program_name = []
    program_start_datetime = []
    program_duration_mins = []

    running_length = sample['ProgramEnd'].hour -  sample['ProgramStart'].hour + 1

    for i in range(running_length):

        program_name.append(sample['Program'])
        program_start_datetime.append(datetime.strptime(str(sample['ProgramStart']), "%Y-%m-%d %H:%M:%S").replace(second=0, microsecond=0, minute=0) + timedelta(hours=i))

        if i == 0:
            program_first_mins = 60 - datetime.strptime(str(sample['ProgramStart']), "%Y-%m-%d %H:%M:%S").minute
            program_duration_mins.append(program_first_mins)
            
        
        elif i != running_length-1:
            program_duration_mins.append(60)
        
        else:
            program_end_mins = datetime.strptime(str(sample['ProgramEnd']), "%Y-%m-%d %H:%M:%S").minute - 0
            program_duration_mins.append(program_end_mins)
    
    return {'program_name':program_name, 'program_start_datetime':program_start_datetime, 'program_duration_mins':program_duration_mins}

    

In [21]:
# modification process
expanded_CNX_program_df = pd.DataFrame()
for i in range(len(CNX_programs_df)):
    expanded_CNX_program_df = pd.concat([expanded_CNX_program_df, pd.DataFrame(expand_program_df(CNX_programs_df.iloc[i]))], axis=0)

expanded_CNX_program_df = expanded_CNX_program_df.rename( \
    columns = {'program_start_datetime':'intervalStart'
               ,'program_name':'CNX_program_name' \
                ,'program_duration_mins':'CNX_program_duration_mins'})

for index, i in enumerate(expanded_CNX_program_df['CNX_program_name'].unique()):
    
    if index == 0:
        CNX_all_program_df = expanded_CNX_program_df[expanded_CNX_program_df['CNX_program_name']==i]
        CNX_all_program_df =CNX_all_program_df.rename(columns={'CNX_program_duration_mins':f'{i}_duration_mins'})
        CNX_all_program_df = CNX_all_program_df.drop(columns=['CNX_program_name'])

    else:
        CNX_one_program_df = expanded_CNX_program_df[expanded_CNX_program_df['CNX_program_name']==i]
        CNX_one_program_df = CNX_one_program_df.rename(columns={'CNX_program_duration_mins':f'{i}_duration_mins'})
        CNX_one_program_df = CNX_one_program_df.drop(columns=['CNX_program_name'])

        CNX_all_program_df = CNX_all_program_df.merge(CNX_one_program_df, on='intervalStart', how='outer')



In [22]:
# modification process
expanded_GRE_program_df = pd.DataFrame()
for i in range(len(GRE_programs_df)):
    expanded_GRE_program_df = pd.concat([expanded_GRE_program_df, pd.DataFrame(expand_program_df(GRE_programs_df.iloc[i]))], axis=0)

    
expanded_GRE_program_df = expanded_GRE_program_df.rename( \
    columns = {'program_start_datetime':'intervalStart'
               ,'program_name':'GRE_program_name' \
                ,'program_duration_mins':'GRE_program_duration_mins'})

for index, i in enumerate(expanded_GRE_program_df['GRE_program_name'].unique()):
    
    if index == 0:
        GRE_all_program_df = expanded_GRE_program_df[expanded_GRE_program_df['GRE_program_name']==i]
        GRE_all_program_df =GRE_all_program_df.rename(columns={'GRE_program_duration_mins':f'{i}_duration_mins'})
        GRE_all_program_df = GRE_all_program_df.drop(columns=['GRE_program_name'])

    else:
        GRE_one_program_df = expanded_GRE_program_df[expanded_GRE_program_df['GRE_program_name']==i]
        GRE_one_program_df = GRE_one_program_df.rename(columns={'GRE_program_duration_mins':f'{i}_duration_mins'})
        GRE_one_program_df = GRE_one_program_df.drop(columns=['GRE_program_name'])

        GRE_all_program_df = GRE_all_program_df.merge(GRE_one_program_df, on='intervalStart', how='outer')

GRE_all_program_df = GRE_all_program_df.filter(['intervalStart', 'Interruptible Irrigation_duration_mins', 'Cycled Air Conditioning_duration_mins', 'Interruptible Water Heating_duration_mins'])

In [23]:
merge_1 = load_weather_df.merge(CNX_all_program_df, on='intervalStart', how='left')
load_weather_programs_df = merge_1.merge(GRE_all_program_df, on='intervalStart', how='left')


In [24]:
load_weather_programs_df.shape

(58634, 33)

In [25]:
# check data aggregated correctly
load_weather_programs_df[(load_weather_programs_df['intervalStart']>'2023-08-24 12:00:00') & \
     (load_weather_programs_df['intervalStart']<'2023-08-25 00:00:00')]\
          [['intervalStart', 'DVR_duration_mins',	'CampusGen_duration_mins', 'CIGen_duration_mins',\
            'ACST_duration_mins',	'PTR_duration_mins', 'Interruptible Irrigation_duration_mins',\
            'Cycled Air Conditioning_duration_mins', 'Interruptible Water Heating_duration_mins']]

,intervalStart,DVR_duration_mins,CampusGen_duration_mins,CIGen_duration_mins,ACST_duration_mins,PTR_duration_mins,Interruptible Irrigation_duration_mins,Cycled Air Conditioning_duration_mins,Interruptible Water Heating_duration_mins
57866,2023-08-24 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57867,2023-08-24 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,30.0,60.0
57868,2023-08-24 15:00:00,60.0,60.0,NaN,60.0,60.0,NaN,60.0,60.0
57869,2023-08-24 16:00:00,60.0,60.0,NaN,60.0,60.0,NaN,60.0,60.0
57870,2023-08-24 17:00:00,60.0,60.0,NaN,60.0,60.0,NaN,60.0,60.0
57871,2023-08-24 18:00:00,0.0,0.0,NaN,60.0,0.0,NaN,60.0,60.0
57872,2023-08-24 19:00:00,NaN,NaN,NaN,0.0,NaN,NaN,60.0,60.0
57873,2023-08-24 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,30.0,60.0
57874,2023-08-24 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
57875,2023-08-24 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
load_weather_programs_df = load_weather_programs_df.drop(['dateTime'], axis=1)

In [27]:
load_weather_programs_df

,intervalStart,Connexus_kWh,unixTime,latitude,longitude,station,apparentTemperature,cloudCover,dewPoint,humidity,...,windGust,windSpeed,DVR_duration_mins,CampusGen_duration_mins,CIGen_duration_mins,ACST_duration_mins,PTR_duration_mins,Interruptible Irrigation_duration_mins,Cycled Air Conditioning_duration_mins,Interruptible Water Heating_duration_mins
0,2017-01-01 01:00:00,194634.421900,1483254000,45.395556,-93.386667,STF,18.93,0.39,19.23,0.77,...,10.51,5.48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-01-01 02:00:00,185003.523400,1483257600,45.395556,-93.386667,STF,20.16,0.00,18.98,0.76,...,8.88,4.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-01-01 03:00:00,179523.593800,1483261200,45.395556,-93.386667,STF,20.40,0.04,18.58,0.77,...,6.34,3.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-01-01 04:00:00,178223.679700,1483264800,45.395556,-93.386667,STF,24.46,0.20,18.62,0.78,...,5.83,2.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-01-01 05:00:00,180018.203100,1483268400,45.395556,-93.386667,STF,23.61,0.11,18.69,0.81,...,3.13,1.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58629,2023-09-25 08:00:00,234042.117832,1695646800,45.395556,-93.386667,STF,60.14,0.99,59.33,0.97,...,7.68,4.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58630,2023-09-25 09:00:00,235179.112163,1695650400,45.395556,-93.386667,STF,59.63,1.00,58.37,0.96,...,12.48,8.43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58631,2023-09-25 10:00:00,237921.409900,1695654000,45.395556,-93.386667,STF,59.27,1.00,58.49,0.98,...,7.61,6.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58632,2023-09-25 11:00:00,240668.883457,1695657600,45.395556,-93.386667,STF,59.05,0.99,58.16,0.97,...,7.05,4.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Replace null values

In [28]:
# check null values
for column in load_weather_programs_df.columns:
    if load_weather_programs_df[column].isnull().sum()!=0:
        print(column, load_weather_programs_df[column].isnull().sum())

cloudCover 14
icon 4
precipAccumulation 40544
precipIntensity 13
precipProbability 13
precipType 42518
pressure 201
snowAccumulation 45023
snowIntensity 45028
uvIndex 8
visibility 9
windBearing 259
windGust 23
DVR_duration_mins 58446
CampusGen_duration_mins 58450
CIGen_duration_mins 58571
ACST_duration_mins 58572
PTR_duration_mins 58606
Interruptible Irrigation_duration_mins 58619
Cycled Air Conditioning_duration_mins 57971
Interruptible Water Heating_duration_mins 57251


In [29]:
# obseve not_null values of columns include null-values
for column in load_weather_programs_df.columns:
    if load_weather_programs_df[column].isnull().sum()!=0:
        print(column, '\n', load_weather_programs_df[column][load_weather_programs_df[column].isnull()==False].value_counts(), '\n')

cloudCover 
 0.00    11293
1.00    11022
0.01     1336
0.02     1026
0.03      968
        ...  
0.52      209
0.60      207
0.70      203
0.76      194
0.73      184
Name: cloudCover, Length: 101, dtype: int64 

icon 
 clear-night               9376
clear-day                 8498
cloudy                    8478
partly-cloudy-day         7831
partly-cloudy-night       6371
Clear                     5629
Cloudy                    4564
rain                      2986
Mostly Clear              1321
snow                       938
Partly Cloudy              647
fog                        465
Mostly Cloudy              293
Light Snow                 277
Flurries                   230
Light Rain                 196
Light Fog                   92
Fog                         91
Drizzle                     79
Rain                        68
Snow                        64
wind                        42
sleet                       33
Heavy Rain                  25
Heavy Snow                  17
Light

## Null value imputation strategy

### cloudCover
0.00 &nbsp;&nbsp; 11293 <br>
1.00 &nbsp;&nbsp; 10992 <br>
0.01 &nbsp;&nbsp; 1336 <br>
0.02 &nbsp;&nbsp; 1026 <br>
0.03 &nbsp;&nbsp; 968

Number of null values: 14/58560 <br>
Using average value. <br>

<br><br>

### icon
clear-night      &nbsp;&nbsp;         9376 <br>
clear-day        &nbsp;&nbsp;         8498 <br>
cloudy           &nbsp;&nbsp;         8478 <br>
partly-cloudy-day    &nbsp;&nbsp;     7831 <br>
partly-cloudy-night  &nbsp;&nbsp;     6371 <br>
Clear          &nbsp;&nbsp;           5629 <br>
Cloudy         &nbsp;&nbsp;           4519 <br>
rain           &nbsp;&nbsp;           2986 <br>
Mostly Clear   &nbsp;&nbsp;           1317 <br>
snow           &nbsp;&nbsp;            938 <br>

Number of null values: 4/58560 <br>
Using the most common value. <br>

<br><br>

### precipAccumulation 
0.00  &nbsp;&nbsp;  13851 <br>
0.01  &nbsp;&nbsp;   1822 <br>
0.02  &nbsp;&nbsp;    763 <br>
0.03  &nbsp;&nbsp;    346 <br>
0.04  &nbsp;&nbsp;    204 <br>

Number of null values: 40544/58560 <br>
Using 0.00 because None value could represent the day withour rain. <br>

<br><br>

### precipIntensity 
0.0000  &nbsp;&nbsp;  43665 <br>
0.0010  &nbsp;&nbsp;    963 <br>
0.0002  &nbsp;&nbsp;    881 <br>
0.0009  &nbsp;&nbsp;    774 <br>
0.0003  &nbsp;&nbsp;    769 <br>

Number of null values: 13/58560 <br>
Using 0.00 because None value could represent the day withour rain. <br>

<br><br>

### precipProbability 
0.00  &nbsp;&nbsp;  43514 <br>
0.01  &nbsp;&nbsp;   3712 <br>
0.02  &nbsp;&nbsp;   1993 <br>
1.00  &nbsp;&nbsp;   1340 <br>
0.03  &nbsp;&nbsp;    978 <br>

Number of null values: 13/58560 <br>
Using 0.00 because None value could represent the day withour rain. <br>

<br><br>

### precipType 
rain  &nbsp;&nbsp;   10663 <br>
snow  &nbsp;&nbsp;    5314 <br>
sleet &nbsp;&nbsp;      87 <br>

Number of null values: 42496/58560 <br>
Using 'no_precip' because None value could represent the day withour rain, snow or sleet. <br>

<br><br>

### pressure 
30.01  &nbsp;&nbsp;  1200 <br>
29.93  &nbsp;&nbsp;  1198 <br>
29.90  &nbsp;&nbsp;  1148 <br>
29.91  &nbsp;&nbsp;  1122 <br>
29.98  &nbsp;&nbsp;  1103 <br>

Number of null values: 201/58560 <br>
Using 'K-nearest algo', find the most similar days as the day include null value pressure. <br>
And replace the null value pressure with the average pressure of top n similar days.  <br>

<br><br>

### snowAccumulation 
0.00  &nbsp;&nbsp;  12801 <br>
0.01  &nbsp;&nbsp;     87 <br>
0.02  &nbsp;&nbsp;     75 <br>
0.04  &nbsp;&nbsp;     43 <br>
0.03  &nbsp;&nbsp;     41 <br>

Number of null values: 45023/58560 <br>
Using 0.00 because None value could represent the day withour snow. <br>

<br><br>

### snowIntensity 
0.000  &nbsp;&nbsp;  13061 <br>
0.010  &nbsp;&nbsp;    103 <br>
0.020  &nbsp;&nbsp;     91 <br>
0.030  &nbsp;&nbsp;     41 <br>
0.040  &nbsp;&nbsp;     27 <br>
0.050  &nbsp;&nbsp;     23 <br>
0.060  &nbsp;&nbsp;     19 <br>

Number of null values: 45028/58560 <br>
Using 0.00 because None value could represent the day withour snow. <br>

<br><br>

### uvIndex 
0.0  &nbsp;&nbsp;   38699 <br>
1.0  &nbsp;&nbsp;    6612 <br>
2.0  &nbsp;&nbsp;    3644 <br>
3.0  &nbsp;&nbsp;    2846 <br>
4.0  &nbsp;&nbsp;    2261 <br>
5.0  &nbsp;&nbsp;    1674 <br>
6.0  &nbsp;&nbsp;    1284 <br>

Number of null values: 8/58560 <br>
Using the most common value. <br>

<br><br>

### visibility 
10.00 &nbsp;&nbsp;   33745 <br>
9.94  &nbsp;&nbsp;   10050 <br>
7.58  &nbsp;&nbsp;     186 <br>
9.98  &nbsp;&nbsp;     143 <br>
9.93  &nbsp;&nbsp;     137 <br>

Number of null values: 9/58560 <br>
Using the most common value. <br>

<br><br>

### windBearing 
300.0  &nbsp;&nbsp;  274 <br>
307.0  &nbsp;&nbsp;  270 <br>
309.0  &nbsp;&nbsp;  268 <br>
292.0  &nbsp;&nbsp; 267 <br>
294.0  &nbsp;&nbsp;  263 <br>

Number of null values: 259/58560 <br>
Using 'K-nearest algo', find the most similar days as the day include null value winBearing. <br>
And replace the null value winBearing with the average winBearing of top n similar days. <br>

<br><br>

### windGust 
13.14  &nbsp;&nbsp;  183 <br>
10.91  &nbsp;&nbsp;  180 <br>
15.24  &nbsp;&nbsp;  163 <br>
10.35  &nbsp;&nbsp;  156 <br>
12.30  &nbsp;&nbsp;  156 <br>

Number of null values: 23/58560 <br>
Using the most common value. <br>


## Imputation strategy summary

### using the most common value: 
CloudCover, icon, uvIndex, visibility, windGust

### K-nearest neighborhood: 
Pressure, windBearing

#### Others:

precipAccumulation: 0.00 <br>
precipIntensity: 0.00 <br>
precipProbability: 0.00 <br>
precipType: no-precip <br>
snowIntensity: 0.00 <br>
snowAccumulation: 0.00 <br>


#### Null values for programs:

Using 0.

## Imputation 

In [30]:
load_weather_programs_df.head()

,intervalStart,Connexus_kWh,unixTime,latitude,longitude,station,apparentTemperature,cloudCover,dewPoint,humidity,...,windGust,windSpeed,DVR_duration_mins,CampusGen_duration_mins,CIGen_duration_mins,ACST_duration_mins,PTR_duration_mins,Interruptible Irrigation_duration_mins,Cycled Air Conditioning_duration_mins,Interruptible Water Heating_duration_mins
0,2017-01-01 01:00:00,194634.4219,1483254000,45.395556,-93.386667,STF,18.93,0.39,19.23,0.77,...,10.51,5.48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-01-01 02:00:00,185003.5234,1483257600,45.395556,-93.386667,STF,20.16,0.00,18.98,0.76,...,8.88,4.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-01-01 03:00:00,179523.5938,1483261200,45.395556,-93.386667,STF,20.40,0.04,18.58,0.77,...,6.34,3.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-01-01 04:00:00,178223.6797,1483264800,45.395556,-93.386667,STF,24.46,0.20,18.62,0.78,...,5.83,2.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-01-01 05:00:00,180018.2031,1483268400,45.395556,-93.386667,STF,23.61,0.11,18.69,0.81,...,3.13,1.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
load_weather_programs_df.columns

Index(['intervalStart', 'Connexus_kWh', 'unixTime', 'latitude', 'longitude',
       'station', 'apparentTemperature', 'cloudCover', 'dewPoint', 'humidity',
       'icon', 'precipAccumulation', 'precipIntensity', 'precipProbability',
       'precipType', 'pressure', 'snowAccumulation', 'snowIntensity',
       'temperature', 'uvIndex', 'visibility', 'windBearing', 'windGust',
       'windSpeed', 'DVR_duration_mins', 'CampusGen_duration_mins',
       'CIGen_duration_mins', 'ACST_duration_mins', 'PTR_duration_mins',
       'Interruptible Irrigation_duration_mins',
       'Cycled Air Conditioning_duration_mins',
       'Interruptible Water Heating_duration_mins'],
      dtype='object')

In [32]:
load_weather_programs_df[['precipAccumulation', 'precipIntensity', 'precipProbability', \
                          'snowAccumulation', 'snowIntensity']] = \
                          load_weather_programs_df[['precipAccumulation', 'precipIntensity', 'precipProbability', \
                          'snowAccumulation', 'snowIntensity']].fillna(value=0.00)

In [33]:
load_weather_programs_df['precipType'] = load_weather_programs_df['precipType'].fillna(value='no_precip')

In [34]:
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder

In [35]:
imp_nan = SimpleImputer(missing_values=np.NaN, strategy='most_frequent')
imp_none = SimpleImputer(missing_values=None, strategy='most_frequent')

In [36]:
load_weather_programs_df['cloudCover'] = imp_nan.fit_transform(load_weather_programs_df[['cloudCover']]).ravel()

load_weather_programs_df['icon'] = imp_nan.fit_transform(load_weather_programs_df[['icon']]).ravel()
load_weather_programs_df['icon'] = imp_none.fit_transform(load_weather_programs_df[['icon']]).ravel()


load_weather_programs_df['uvIndex'] = imp_nan.fit_transform(load_weather_programs_df[['uvIndex']]).ravel()

load_weather_programs_df['visibility'] = imp_nan.fit_transform(load_weather_programs_df[['visibility']]).ravel()

load_weather_programs_df['windGust'] = imp_nan.fit_transform(load_weather_programs_df[['windGust']]).ravel()


In [37]:
for program in load_weather_programs_df.columns[24:]:

    load_weather_programs_df[program] = load_weather_programs_df[program].fillna(value=0)

In [38]:
# check null values
for column in load_weather_programs_df.columns:
    if load_weather_programs_df[column].isnull().sum()!=0:
        print(column, load_weather_programs_df[column].isnull().sum())

pressure 201
windBearing 259


In [39]:
load_weather_programs_df.head()
# take out programs
# run..


,intervalStart,Connexus_kWh,unixTime,latitude,longitude,station,apparentTemperature,cloudCover,dewPoint,humidity,...,windGust,windSpeed,DVR_duration_mins,CampusGen_duration_mins,CIGen_duration_mins,ACST_duration_mins,PTR_duration_mins,Interruptible Irrigation_duration_mins,Cycled Air Conditioning_duration_mins,Interruptible Water Heating_duration_mins
0,2017-01-01 01:00:00,194634.4219,1483254000,45.395556,-93.386667,STF,18.93,0.39,19.23,0.77,...,10.51,5.48,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2017-01-01 02:00:00,185003.5234,1483257600,45.395556,-93.386667,STF,20.16,0.00,18.98,0.76,...,8.88,4.40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-01-01 03:00:00,179523.5938,1483261200,45.395556,-93.386667,STF,20.40,0.04,18.58,0.77,...,6.34,3.59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2017-01-01 04:00:00,178223.6797,1483264800,45.395556,-93.386667,STF,24.46,0.20,18.62,0.78,...,5.83,2.88,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2017-01-01 05:00:00,180018.2031,1483268400,45.395556,-93.386667,STF,23.61,0.11,18.69,0.81,...,3.13,1.66,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


For knn imputation we need to convert categorical data to numeric data, because knn calculate distance between two samples to find the most closest(similar) samples.   

In [40]:
load_weather_programs_df.describe(include=object).columns

Index(['station', 'icon', 'precipType'], dtype='object')

In [41]:
category_columns = load_weather_programs_df.describe(include=object).columns

In [42]:
le = LabelEncoder()

for one_column in category_columns:

    label = le.fit_transform(load_weather_programs_df[one_column])

    load_weather_programs_df[one_column] = label

In [43]:
knn_imputer = KNNImputer(n_neighbors=5)

In [44]:
rest_columns = load_weather_programs_df.drop(['intervalStart'], axis=1).columns

load_weather_programs_df[rest_columns] = \
    knn_imputer.fit_transform(load_weather_programs_df.drop(['intervalStart'], axis=1))

In [45]:
load_weather_programs_df

,intervalStart,Connexus_kWh,unixTime,latitude,longitude,station,apparentTemperature,cloudCover,dewPoint,humidity,...,windGust,windSpeed,DVR_duration_mins,CampusGen_duration_mins,CIGen_duration_mins,ACST_duration_mins,PTR_duration_mins,Interruptible Irrigation_duration_mins,Cycled Air Conditioning_duration_mins,Interruptible Water Heating_duration_mins
0,2017-01-01 01:00:00,194634.421900,1.483254e+09,45.395556,-93.386667,0.0,18.93,0.39,19.23,0.77,...,10.51,5.48,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2017-01-01 02:00:00,185003.523400,1.483258e+09,45.395556,-93.386667,0.0,20.16,0.00,18.98,0.76,...,8.88,4.40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-01-01 03:00:00,179523.593800,1.483261e+09,45.395556,-93.386667,0.0,20.40,0.04,18.58,0.77,...,6.34,3.59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2017-01-01 04:00:00,178223.679700,1.483265e+09,45.395556,-93.386667,0.0,24.46,0.20,18.62,0.78,...,5.83,2.88,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2017-01-01 05:00:00,180018.203100,1.483268e+09,45.395556,-93.386667,0.0,23.61,0.11,18.69,0.81,...,3.13,1.66,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58629,2023-09-25 08:00:00,234042.117832,1.695647e+09,45.395556,-93.386667,0.0,60.14,0.99,59.33,0.97,...,7.68,4.76,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58630,2023-09-25 09:00:00,235179.112163,1.695650e+09,45.395556,-93.386667,0.0,59.63,1.00,58.37,0.96,...,12.48,8.43,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58631,2023-09-25 10:00:00,237921.409900,1.695654e+09,45.395556,-93.386667,0.0,59.27,1.00,58.49,0.98,...,7.61,6.06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58632,2023-09-25 11:00:00,240668.883457,1.695658e+09,45.395556,-93.386667,0.0,59.05,0.99,58.16,0.97,...,7.05,4.45,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
# check data aggregated correctly
load_weather_programs_df[(load_weather_programs_df['intervalStart']>'2023-08-24 12:00:00') & \
     (load_weather_programs_df['intervalStart']<'2023-08-25 00:00:00')]\
          [['intervalStart', 'DVR_duration_mins',	'CampusGen_duration_mins', 'CIGen_duration_mins',\
            'ACST_duration_mins',	'PTR_duration_mins', 'Interruptible Irrigation_duration_mins',\
            'Cycled Air Conditioning_duration_mins', 'Interruptible Water Heating_duration_mins']]

,intervalStart,DVR_duration_mins,CampusGen_duration_mins,CIGen_duration_mins,ACST_duration_mins,PTR_duration_mins,Interruptible Irrigation_duration_mins,Cycled Air Conditioning_duration_mins,Interruptible Water Heating_duration_mins
57866,2023-08-24 13:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
57867,2023-08-24 14:00:00,0.0,0.0,0.0,0.0,0.0,0.0,30.0,60.0
57868,2023-08-24 15:00:00,60.0,60.0,0.0,60.0,60.0,0.0,60.0,60.0
57869,2023-08-24 16:00:00,60.0,60.0,0.0,60.0,60.0,0.0,60.0,60.0
57870,2023-08-24 17:00:00,60.0,60.0,0.0,60.0,60.0,0.0,60.0,60.0
57871,2023-08-24 18:00:00,0.0,0.0,0.0,60.0,0.0,0.0,60.0,60.0
57872,2023-08-24 19:00:00,0.0,0.0,0.0,0.0,0.0,0.0,60.0,60.0
57873,2023-08-24 20:00:00,0.0,0.0,0.0,0.0,0.0,0.0,30.0,60.0
57874,2023-08-24 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
57875,2023-08-24 22:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
load_weather_programs_df.to_csv('./load_weather_programs_df.csv')